image.png

In [1]:
import tensorflow as tf
import numpy as np # 추가
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report # 추가


2025-11-23 11:38:12.025275: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 11:38:15.029418: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-23 11:38:28.731796: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 사용자 설정 (원래 값 사용)
train_dir = '/workspace/user4/cropped/train'
val_dir = '/workspace/user4/cropped/val'
target_size = (224, 224)
batch_size = 32
seed = 42

# helper: 디렉터리를 스캔해서 파일경로와 라벨(1:happy, 0:others) DataFrame 생성
def make_dataframe_from_dir(base_dir, exclude_folders=('invalid',), positive_class='happy'):
    rows = []
    # 각 서브폴더(클래스) 검사
    for class_name in sorted(os.listdir(base_dir)):
        class_path = os.path.join(base_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        if class_name in exclude_folders:
            continue
        # 라벨 결정: 'happy' -> 1, others -> 0
        label = 1 if class_name == positive_class else 0
        for fname in sorted(os.listdir(class_path)):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                rows.append({
                    'filepath': os.path.join(class_path, fname),
                    'label': label,
                    'orig_class': class_name
                })
    df = pd.DataFrame(rows)
    return df

# DataFrame 생성
train_df = make_dataframe_from_dir(train_dir, exclude_folders=('invalid',), positive_class='happy')
val_df = make_dataframe_from_dir(val_dir, exclude_folders=('invalid',), positive_class='happy')

print("Train samples:", len(train_df), "Val samples:", len(val_df))
print("Train class distribution:\n", train_df['label'].value_counts())
print("Val class distribution:\n", val_df['label'].value_counts())

# ImageDataGenerator (기존 전처리/증강 사용)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# flow_from_dataframe 생성 (class_mode='raw' or 'binary' 가능; raw returns label array as-is)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='raw',    # 이진 레이블을 그대로 반환 (shape=(batch_size,))
    shuffle=True,
    seed=seed
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='raw',
    shuffle=False,  # 평가 시 순서 유지
    seed=seed
)

# 필요 시 클래스 인덱스(여기서는 이미 0/1로 라벨링 됨)를 확인
# flow_from_dataframe는 .class_indices를 만들지 않으므로 대체 정보 출력
print("Generators created. Example batch shapes:")
x_batch, y_batch = next(train_generator)
print("x:", x_batch.shape, "y:", y_batch.shape, "y unique:", np.unique(y_batch))



Train samples: 9550 Val samples: 1151
Train class distribution:
 label
1    5245
0    4305
Name: count, dtype: int64
Val class distribution:
 label
0    852
1    299
Name: count, dtype: int64
Found 9550 validated image filenames.
Found 1151 validated image filenames.
Generators created. Example batch shapes:
x: (32, 224, 224, 3) y: (32,) y unique: [0 1]


In [ ]:
 
# Base model 로드
base_model = EfficientNetB0(weights='imagenet', include_top=False,
                           input_shape=(target_size[0], target_size[1], 3))

# top 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # 이진 분류

model = Model(inputs=base_model.input, outputs=predictions)

# base freeze
for layer in base_model.layers:
    layer.trainable = False
    
# 컴파일 (이진 교차엔트로피)
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

model.summary()


# 안전한 step 계산
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)

# 학습: 
print("\n--- 모델 학습 시작 ---")
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps,    
)
print("--- 모델 학습 완료 ---\n")

# --- 검증 데이터셋에 대한 성능 평가 지표 추가 ---

print("--- 검증 데이터셋 성능 평가 시작 ---") # 이하 수정



I0000 00:00:1763898286.655148  540278 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 40045 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:53:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,213,668 (16.07 MB)

 Trainable params: 164,097 (641.00 KB)

...

2025-11-23 11:47:18.611158: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 11:47:18.736528: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 11:47:19.107700: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 11:47:19.234456: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 11:47:19.739190: E external/local_xla/xla/stream_

299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 737ms/step - accuracy: 0.7499 - auc: 0.8261 - loss: 0.4918

2025-11-23 11:49:32.389302: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 11:49:32.516617: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 11:49:32.912569: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 11:49:33.039855: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-23 11:49:33.554170: E external/local_xla/xla/stream_

299/299 ━━━━━━━━━━━━━━━━━━━━ 285s 861ms/step - accuracy: 0.8166 - auc: 0.9008 - loss: 0.3990 - val_accuracy: 0.7785 - val_auc: 0.9411 - val_loss: 0.5067
Epoch 2/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 204s 684ms/step - accuracy: 0.8715 - auc: 0.9439 - loss: 0.3017 - val_accuracy: 0.8697 - val_auc: 0.9528 - val_loss: 0.3250
Epoch 3/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 213s 711ms/step - accuracy: 0.8870 - auc: 0.9538 - loss: 0.2742 - val_accuracy: 0.8914 - val_auc: 0.9564 - val_loss: 0.2651
Epoch 4/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 171s 570ms/step - accuracy: 0.8926 - auc: 0.9556 - loss: 0.2673 - val_accuracy: 0.9079 - val_auc: 0.9594 - val_loss: 0.2285
Epoch 5/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 215s 718ms/step - accuracy: 0.8924 - auc: 0.9585 - loss: 0.2596 - val_accuracy: 0.8905 - val_auc: 0.9621 - val_loss: 0.2687
Epoch 6/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 220s 736ms/step - accuracy: 0.8963 - auc: 0.9616 - loss: 0.2491 - val_accuracy: 0.9166 - val_auc: 0.9595 - val_loss: 0.2230
Epoch 7/10
299/299 ━━━━━━━━

AttributeError: 'DataFrameIterator' object has no attribute 'classes'

In [7]:
# --- 검증 데이터셋에 대한 성능 평가 지표 추가 ---
print("--- 검증 데이터셋 성능 평가 시작 ---")

# 1. 검증 데이터셋의 실제 레이블 (True Labels) 추출
if hasattr(validation_generator, 'labels'):
    validation_labels = validation_generator.labels
elif hasattr(validation_generator, 'classes'):
    validation_labels = validation_generator.classes
else:
    validation_labels = val_df['label'].values

# 2. 모델 예측 (Predictions) 수행
validation_generator.reset()
pred_probabilities = model.predict(validation_generator, steps=validation_steps, verbose=1)

# 3. 예측 확률을 이진 클래스 (0 또는 1)로 변환
pred_classes = (pred_probabilities > 0.5).astype(int).flatten()

# 4. classification_report 출력
class_names = ['others', 'happy']
report = classification_report(validation_labels, pred_classes, target_names=class_names, digits=4)
print("\n--- Classification Report (Validation Set) ---\n")
print(report)
print("--- 검증 데이터셋 성능 평가 완료 ---")



--- 검증 데이터셋 성능 평가 시작 ---
36/36 ━━━━━━━━━━━━━━━━━━━━ 26s 550ms/step

--- Classification Report (Validation Set) ---

              precision    recall  f1-score   support

      others     0.9768    0.8897    0.9312       852
       happy     0.7493    0.9398    0.8338       299

    accuracy                         0.9027      1151
   macro avg     0.8631    0.9147    0.8825      1151
weighted avg     0.9177    0.9027    0.9059      1151

--- 검증 데이터셋 성능 평가 완료 ---


In [6]:

# 5. classification_report를 사용하여 지표 계산 및 출력
# precision, recall, f1-score, support를 포함합니다.
report = classification_report(
    validation_labels, 
    pred_classes, 
    target_names=class_names, 
    digits=4 # 소수점 자릿수 지정
)

print("\n--- Classification Report (Validation Set) ---\n")
print(report)

NameError: name 'validation_labels' is not defined